## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
# imports
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
# load data into dataframe
json_df = pd.read_json('data/world_bank_projects.json')

## 1. Find the 10 countries with most projects

**Questions**
- Which column(s) identify the country/countries?
- Can one record include more than one country?
- Does each record correspond to one and only one project? (can I simply count the records/rows to get a count of projects?)
    - *Yes, both `_id` and `id` are unique project IDs*

**Plan**
1. Explore dataframe to identify country column(s)
2. Determine unique ID for country
    - May involve counting nested fields (normalize?)
3. Determine unique ID for project
4. Use something like: `df.groupby('country').value_counts()` to get count
5. Take top 10, save as list

In [3]:
json_df.columns

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url'],
      dtype='object')

In [88]:
# Show all rows/columns in output
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [14]:
json_df.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,docty,envassesmentcategorycode,grantamt,ibrdcommamt,id,idacommamt,impagency,lendinginstr,lendinginstrtype,lendprojectcost,majorsector_percent,mjsector_namecode,mjtheme,mjtheme_namecode,mjthemecode,prodline,prodlinetext,productlinetype,project_abstract,project_name,projectdocs,projectfinancialtype,projectstatusdisplay,regionname,sector,sector1,sector2,sector3,sector4,sector_namecode,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,"Project Information Document,Indigenous People...",C,0,0,P129828,130000000,MINISTRY OF EDUCATION,Investment Project Financing,IN,550000000,"[{'Percent': 46, 'Name': 'Education'}, {'Perce...","[{'code': 'EX', 'name': 'Education'}, {'code':...",[Human development],"[{'code': '8', 'name': 'Human development'}, {...","8,11",PE,IBRD/IDA,L,{'cdata': 'The development objective of the Se...,Ethiopia General Education Quality Improvement...,"[{'DocDate': '28-AUG-2013', 'EntityID': '09022...",IDA,Active,Africa,"[{'Name': 'Primary education'}, {'Name': 'Seco...","{'Percent': 46, 'Name': 'Primary education'}","{'Percent': 26, 'Name': 'Secondary education'}","{'Percent': 16, 'Name': 'Public administration...","{'Percent': 12, 'Name': 'Tertiary education'}","[{'code': 'EP', 'name': 'Primary education'}, ...","ET,BS,ES,EP",IBRD,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,"Project Information Document,Integrated Safegu...",C,4700000,0,P144674,0,MINISTRY OF FINANCE,Specific Investment Loan,IN,5700000,"[{'Percent': 70, 'Name': 'Public Administratio...","[{'code': 'BX', 'name': 'Public Administration...","[Economic management, Social protection and ri...","[{'code': '1', 'name': 'Economic management'},...","1,6",RE,Recipient Executed Activities,L,NaN,TN: DTF Social Protection Reforms Support,"[{'DocDate': '29-MAR-2013', 'EntityID': '00033...",OTHER,Active,Middle East and North Africa,[{'Name': 'Public administration- Other social...,"{'Percent': 70, 'Name': 'Public administration...","{'Percent': 30, 'Name': 'General public admini...",NaN,NaN,"[{'code': 'BS', 'name': 'Public administration...","BZ,BS",IBRD,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,"Resettlement Plan,Environmental Assessment,Int...",B,0,0,P145310,6060000,MINISTRY OF TRANSPORT AND COMMUNICATIONS,Investment Project Financing,IN,6060000,"[{'Percent': 100, 'Name': 'Transportation'}]","[{'code': 'TX', 'name': 'Transportation'}]","[Trade and integration, Public sector governan...","[{'code': '5', 'name': 'Trade and integration'...","5,2,11,6",PE,IBRD/IDA,L,NaN,Tuvalu Aviation Investment Project - Additiona...,"[{'DocDate': '21-OCT-2013', 'EntityID': '00033...",IDA,Active,East Asia and Pacific,[{'Name': 'Rural and Inter-Urban Roads and Hig...,"{'Percent': 100, 'Name': 'Rural and Inter-Urba...",NaN,NaN,NaN,"[{'code': 'TI', 'name': 'Rural and Inter-Urban...",TI,IBRD,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
3,{'$oid': '52b213b38594d8a2be17

**Column Notes**
- `_id` seems like an internal data management id, not really a project ID that would appear on paper somewhere
- `borrower` and `country_x` fields seem promising for country codes
- `id` looks like a project ID
    - do a `value_counts` for this field and compare to `_id`
- `projectstatusdisplay` can be used to group Active vs Inactive (or some other code?) projects
    - how does this compare to `status` column?
- `regionname` could be useful for identifying outliers/errors in region/country coding


In [103]:
# Country Columns
country_cols = ['country_namecode',
                'countrycode',
                'countryname',
                'countryshortname']


In [68]:
# convert '_id' column (dict) into a string column '$oid'
json_df = pd.concat([json_df, json_df['_id'].apply(pd.Series)], axis=1)

In [69]:
# confirm that '$oid' was added
json_df.columns

Index(['_id', 'approvalfy', 'board_approval_month', 'boardapprovaldate',
       'borrower', 'closingdate', 'country_namecode', 'countrycode',
       'countryname', 'countryshortname', 'docty', 'envassesmentcategorycode',
       'grantamt', 'ibrdcommamt', 'id', 'idacommamt', 'impagency',
       'lendinginstr', 'lendinginstrtype', 'lendprojectcost',
       'majorsector_percent', 'mjsector_namecode', 'mjtheme',
       'mjtheme_namecode', 'mjthemecode', 'prodline', 'prodlinetext',
       'productlinetype', 'project_abstract', 'project_name', 'projectdocs',
       'projectfinancialtype', 'projectstatusdisplay', 'regionname', 'sector',
       'sector1', 'sector2', 'sector3', 'sector4', 'sector_namecode',
       'sectorcode', 'source', 'status', 'supplementprojectflg', 'theme1',
       'theme_namecode', 'themecode', 'totalamt', 'totalcommamt', 'url',
       '$oid'],
      dtype='object')

In [79]:
# max of 'id' and '$oid' value counts
# tells me if there are duplicates
print('max of json_df[\'id\']: ' + str(json_df['id'].value_counts().max()))
print('max of json_df[\'$oid\']: ' + str(json_df['$oid'].value_counts().max()))

max of json_df['id']: 1
max of json_df['$oid']: 1


Appears that both `[id]` and `[$oid]` are unique counts of project

In [124]:
#json_df[country_cols]
#country_cols
#json_df['countrycode'].value_counts()
#json_df['countryshortname'].value_counts()
json_df.groupby(['countrycode', 'countryshortname']).size()

countrycode  countryshortname                      
1W           World                                      2
3A           Africa                                    11
4E           East Asia and Pacific                      1
4P           Pacific Islands                            4
5M           Middle East and North Africa               4
7E           Europe and Central Asia                    1
8S           South Asia                                 2
AF           Afghanistan                                6
AG           Antigua and Barbuda                        1
AL           Albania                                    4
AM           Armenia                                    8
AO           Angola                                     1
AR           Argentina                                  2
AZ           Azerbaijan                                 4
BA           Bosnia and Herzegovina                     1
BD           Bangladesh                                12
BF           Burkina

Values of `['1W', '3A', '4E', '4P', '5M', '7E', '8S']` in `'countrycode'` are not valid countries, they are regions
- look at `'borrower'` column for these records

In [136]:
#json_df['countrycode']# in ['1W', '3A', '4E', '4P', '5M', '7E', '8S']
#json_df[json_df['countrycode'] == '1W']
json_df[json_df['countrycode'].isin(['1W', '3A', '4E', '4P', '5M', '7E', '8S'])]

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,docty,envassesmentcategorycode,grantamt,ibrdcommamt,id,idacommamt,impagency,lendinginstr,lendinginstrtype,lendprojectcost,majorsector_percent,mjsector_namecode,mjtheme,mjtheme_namecode,mjthemecode,prodline,prodlinetext,productlinetype,project_abstract,project_name,projectdocs,projectfinancialtype,projectstatusdisplay,regionname,sector,sector1,sector2,sector3,sector4,sector_namecode,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url,$oid
25,{'$oid': '52b213b38594d8a2be17c799'},2014,September,2013-09-30T00:00:00Z,UNIVERSITY OF QUEENSLAND,NaN,East Asia and Pacific!$!4E,4E,East Asia and Pacific,East Asia and Pacific,"Project Paper,Project Information Document,Int...",B,4500000,0,P123933,0,UNIVERSITY OF QUEENSLAND,Investment Project Financing,IN,6500000,"[{'Percent': 50, 'Name': 'Agriculture, fishing...","[{'code': 'AX', 'name': 'Agriculture, fishing,...","[Environment and natural resources management,...","[{'code': '11', 'name': 'Environment and natur...","11,11,11",GE,Global Environment Project,L,{'cdata': 'The development objective of the Ca...,Capturing Coral Reef Ecosystem Services (CCRES),"[{'DocDate': '04-SEP-2013', 'EntityID': '00035...",OTHER,Active,East Asia and Pacific,"[{'Name': 'General agriculture, fishing and fo...","{'Percent': 50, 'Name': 'General agriculture, ...","{'Percent': 50, 'Name': 'Agricultural extensio...",NaN,NaN,"[{'code': 'AZ', 'name': 'General agriculture, ...","AB,AZ",IBRD,Active,N,"{'Percent': 40, 'Name': 'Other environment and...","[{'code': '86', 'name': 'Other environment and...","80,81,86",0,4500000,http://www.worldbank.org/projects/P123933/capt...,52b213b38594d8a2be17c799
27,{'$oid': '52b213b38594d8a2be17c79b'},2014,September,2013-09-30T00:00:00Z,PACIFIC AVIATION SECURITY OFFICE,2018-12-31T00:00:00Z,Pacific Islands!$!4P,4P,Pacific Islands,Pacific Islands,"Project Appraisal Document,Project Information...",C,0,0,P145057,2150000,PACIFIC AVIATION SECURITY OFFICE,Investment Project Financing,IN,2150000,"[{'Percent': 85, 'Name': 'Transportation'}, {'...","[{'code': 'TX', 'name': 'Transportation'}, {'c...","[Environment and natural resources management,...","[{'code': '11', 'name': 'Environment and natur...","11,2,5,6",PE,IBRD/IDA,L,{'cdata': 'The development objective of the Pa...,Pacific Aviation Safety Office Reform,"[{'DocDate': '03-SEP-2013', 'EntityID': '00035...",IDA,Active,East Asia and Pacific,"[{'Name': 'Aviation'}, {'Name': 'Public admini...","{'Percent': 85, 'Name': 'Aviation'}","{'Percent': 15, 'Name': 'Public administration...",NaN,NaN,"[{'code': 'TV', 'name': 'Aviation'}, {'code': ...","BV,TV",IBRD,Active,N,"{'Percent': 5, 'Name': 'Climate change'}","[{'code': '81', 'name': 'Climate change'}, {'c...","52,47,25,81",2150000,2150000,http://www.worldbank.org/projects/P145057/paci...,52b213b38594d8a2be17c79b
45,{'$oid': '52b213b38594d8a2be17c7ad'},2014,September,2013-09-12T00:00:00Z,ECOWAS,NaN,Africa!$!3A,3A,Africa,Africa,"Project Information Document,Integrated Safegu...",C,10000000,0,P125018,0,WAHO,Investment Project Financing,IN,10750000,"[{'Percent': 100, 'Name': 'Health and other so...","[{'code': 'JX', 'name': 'Health and other soci...",[Human development],"[{'code': '8', 'name': 'Human development'}, {...","8,10",RE,Recipient Executed Activities,L,NaN,West Africa Regional Disease Surveillance Cap...,"[{'DocDate': '28-MAR-2013', 'EntityID': '09022...",OTHER,Active,Africa,[{'Name': 'Health'}],"{'Percent': 100, 'Name': 'Health'}",NaN,NaN,NaN,"[{'code': 'JA', 'name': 'Health'}]",JA,IBRD,Active,N,"{'Percent': 100, 'Name': 'Health system perfor...","[{'code': '67', 'name': 'Health system perform...",67,0,10000000,http://www.worldbank.org/projects/P125018/west...,52b213b38594d8a2be17c7ad
46,{'$oid': '52b213b38594d8a2be17c7ae'},2014,September,2013-09-10T00:00:00Z,UGANDA-COMOROS,2018-06-30T00:00:00Z,Africa!$